In [21]:
import dash
from dash import dcc, html, Input, Output, dash_table
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Load poverty data
poverty_data = pd.read_csv("data/clean_data/state_poverty.csv")
poverty_data["code"] = poverty_data["code"].str.upper()

# Create US map with poverty rates
fig_map = px.choropleth(
    poverty_data,
    locations="code",
    locationmode="USA-states",
    color="total_pov",
    hover_name="name",
    color_continuous_scale="Reds",
    scope="usa",
    title="Click a State to See Substance Use vs Poverty"
)

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server  # for deployment

# Layout
app.layout = html.Div([
    html.H2("US Substance Use & Poverty Dashboard"),
    dcc.Graph(id="us-map", figure=fig_map),
    dcc.Graph(id="state-details"),
    html.H4("Raw CSV Data:"),
    dash_table.DataTable(
        id="raw-data-table",
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left"},
        page_size=10
    )
])

# Callback to update graph and table
@app.callback(
    Output("state-details", "figure"),
    Output("raw-data-table", "data"),
    Output("raw-data-table", "columns"),
    Input("us-map", "clickData")
)
def update_state_plot(clickData):
    if not clickData:
        return (
            go.Figure().update_layout(title="Click on a state to view data."),
            [], []
        )

    state_code = clickData["points"][0]["location"]
    state_name = poverty_data[poverty_data["code"] == state_code]["name"].values[0]
    file_path = f"data/state_data/{state_name.lower()}.csv"

    if not os.path.exists(file_path):
        return (
            go.Figure().update_layout(title=f"No drug usage data found for {state_name}"),
            [], []
        )

    # Load drug usage data
    df = pd.read_csv(file_path)
    df = df.iloc[8:18, :].drop(columns=["Unnamed: 0"], errors="ignore")
    df = df.rename(columns={df.columns[0]: "Age Group"})
    df["state"] = state_name
    drug_columns = df.columns[2:-1]
    df_melted = df.melt(id_vars=["Age Group"], value_vars=drug_columns,
                        var_name="Substance", value_name="Usage (%)")
    state_pov = poverty_data[poverty_data["name"] == state_name]

    # Create subplot
    subfig = make_subplots(
        rows=1, cols=2,
        column_widths=[0.75, 0.25],
        specs=[[{"type": "xy"}, {"type": "xy"}]],
        subplot_titles=[f"{state_name} - Substance Use by Age", "Poverty Rates"]
    )

    for substance in df_melted["Substance"].unique():
        df_sub = df_melted[df_melted["Substance"] == substance]
        subfig.add_trace(
            go.Bar(x=df_sub["Age Group"], y=df_sub["Usage (%)"], name=substance),
            row=1, col=1
        )

    if not state_pov.empty:
        subfig.add_trace(
            go.Bar(
                x=["Total Poverty", "Child Poverty"],
                y=[state_pov.iloc[0]["total_pov"], state_pov.iloc[0]["child_pov"]],
                marker_color="crimson",
                name="Poverty"
            ),
            row=1, col=2
        )

    subfig.update_layout(
        title=f"{state_name}: Drug Use vs Poverty",
        height=600,
        width=1200,
        showlegend=True,
        legend_title="Drug Type",
        legend=dict(
            orientation="v",
            x=1.02,
            y=1,
            bgcolor="rgba(255,255,255,0.7)",
            bordercolor="black",
            borderwidth=1
        )
    )
    subfig.update_xaxes(title_text="Age Groups", row=1, col=1)
    subfig.update_yaxes(title_text="Usage (%)", row=1, col=1)
    subfig.update_yaxes(title_text="Poverty Rate (%)", row=1, col=2)

    # Format raw CSV as table
    table_data = df.to_dict("records")
    table_columns = [{"name": col, "id": col} for col in df.columns]

    return subfig, table_data, table_columns

# Run the app
if __name__ == "__main__":
    app.run(debug=True, port=8051)
